# MNIST: multilayer perceptrons

Multilayer perceptrons (MLP) have been around for more than 25 years. They are suited to both regression and classification tasks. Here we will use a classic MLP to recognize handwritten digits (MNIST data set), as well as an MLP with a number of more recent twists.

## Data preparation

### Required imports

In [ ]:
import keras
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

### Obtaining the dataset

In Keras' datasets module we have a handle to the MNIST dataset we want to use in this notebook.  Download the training and test set for this data.

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Before we can start doing machine learning on the data, some preparations are required.  In this case, the data set is clean, which simplifies this step considerably.  Although it would be better to create a pipeline using the scikit-learn framework, we'll do the preparation by hand in this case.

### Transforming the data

Rather than use the 28 $\times$ 28 images as input directly, we reshape each image to a 784 array.

In [ ]:
input_reshaper = FunctionTransformer(lambda x: x.reshape(x.shape[0], -1),
                                     validate=False)

Most training algorithms work better if the input is normalized between 0 and 1.

In [ ]:
input_normalizer = FunctionTransformer(lambda x: x.astype(np.float32)/255.0,
                                       validate=True)

Both preprocessing steps can be combined into a single pipeline.

In [ ]:
input_pipeline = Pipeline([
    ('reshaper', input_reshaper),
    ('normalizer', input_normalizer),
])

We can now fit and transform the training input, and transform the testing input.

In [ ]:
x_train = input_pipeline.fit_transform(x_train)
x_test = input_pipeline.transform(x_test)

The output can be transformed to categorical data, i.e., one category for each digit, rather than a `uint8`. This is a one-hot encoding, so the output is now an array consisting of a single 1.0 value, and nine 0.0 values.  Before calling the `OneHotEncoder`, the output array has to be reshaped.  Note that the type of the output is now `float64`, which we can reduce to `float32` to reduce memory requirements and speed up computations.

In [ ]:
output_reshaper = FunctionTransformer(lambda x: x.reshape(-1, 1),
                                      validate=False)

In [ ]:
output_encoder = OneHotEncoder(categories='auto')

In [ ]:
output_type_changer = FunctionTransformer(lambda x: x.astype(np.float32),
                                          validate=False)

In [ ]:
output_pipeline = Pipeline([
    ('reshaper', output_reshaper),
    ('binarizer', output_encoder),
    ('type_changer', output_type_changer),
])

Now the training output can be fit and transformed, and the testing output transformed accordingly.

In [ ]:
y_train = output_pipeline.fit_transform(y_train)
y_test = output_pipeline.transform(y_test)

In [ ]:
y_train.shape

In [ ]:
print(y_train[0])

Note that the encoding of the output is now a sparse array.

### Validation set

In order to make this reproducible, we have to seed the random number generator.

In [ ]:
np.random.seed(1234)

During the training, we will require a validation set, so we split the training data into two sets, one for actual training, the other for validation.  Note, we don't touch the test data set at all during the training process.  The default is using 75 % of the data for training, 25 % for validation. This function will also shuffle the data set prior to splitting (hence seeding the random number generator).

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train)

In [ ]:
x_train.shape, x_val.shape, y_train.shape, y_val.shape

### Verification

Since we've done quite some transformations, let's verify whether we didn't mess up.

In [ ]:
frame = plt.gca()
frame.axes.get_xaxis().set_visible(False)
frame.axes.get_yaxis().set_visible(False)
plt.imshow(x_train[0].reshape(28, 28), cmap='gray');

In [ ]:
print(y_train[0])

Everything seems fine, input/output are as expected, and we can start doing some machine learning.

## Classic multilayer perceptron (MLP)

We start off training a classic multilayer neural network to familiarie ourselves with the keras framework.

### Required imports

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.optimizers import SGD
from sklearn.metrics import confusion_matrix
import tensorflow as tf

### Model definition

We will create a fully connected neural network with 784 input units (28 $\times$ 28 pixels), two hidden layers with 512 units each, and an output layter with 10 units (ten categories, one per digit). For the two hidden layers, we will use a ReLU activation function, and a SoftMax for the output layer.

To obtain repeatable results, we need to seed the random number generator, taking into account that TensorFlow uses its own.

In [ ]:
np.random.seed(1234)
tf.set_random_seed(4958)

In [ ]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

Now we can can compile the model, specifying the loss function (categorical cross-entropy), the optimizer (SGD, Stochastic Gradient Descent), and the metrics (accuracy) we want to use.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(), metrics=['accuracy'])

### Training

In [ ]:
model_history = model.fit(x_train, y_train, batch_size=128, epochs=100,
                          verbose=1, validation_data=(x_val, y_val))

Plot the history of the training process.

In [ ]:
def plot_history(network_history):
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(network_history.history['loss'])
    plt.plot(network_history.history['val_loss'])
    plt.legend(['Training', 'Validation'])

    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(network_history.history['acc'])
    plt.plot(network_history.history['val_acc'])
    plt.legend(['Training', 'Validation'], loc='lower right')

In [ ]:
plot_history(model_history)

Let's compare the performance of the model on the training, validation and test set.

In [ ]:
model.evaluate(x_train, y_train)

In [ ]:
model.evaluate(x_val, y_val)

In [ ]:
model.evaluate(x_test, y_test)

### Save model

We can save the model as an HDF5 file so that it can later be reloaded.

In [ ]:
model.save('mnist_mlp.h5')

### Dropout layers

In order to reduce overfitting, drop out can be applied, i.e., randomly setting input values to 0 during training. We add a drop-out layer between the hidden layers, and between the last hidden layer and the output layer.

In [ ]:
np.random.seed(1234)

In [ ]:
dropout_model = Sequential()
dropout_model.add(Dense(512, activation='relu', input_shape=(784,)))
dropout_model.add(Dropout(0.2))
dropout_model.add(Dense(512, activation='relu'))
dropout_model.add(Dropout(0.2))
dropout_model.add(Dense(10, activation='softmax'))

In [ ]:
dropout_model.summary()

In [ ]:
dropout_model.compile(loss='categorical_crossentropy', optimizer=SGD(),
                      metrics=['accuracy'])

In [ ]:
dropout_model_history = dropout_model.fit(x_train, y_train, batch_size=128,
                                          epochs=100, verbose=1,
                                          validation_data=(x_val, y_val))

In [ ]:
plot_history(dropout_model_history)

Again, let's compare the performance of the model on the training, validation and test set.

In [ ]:
dropout_model.evaluate(x_train, y_train)

In [ ]:
dropout_model.evaluate(x_val, y_val)

In [ ]:
dropout_model.evaluate(x_test, y_test)

In [ ]:
dropout_model.save('mnist_mlp_dropout.h5')

## Understanding the model

We should try to gain some insight into the model.  There are several ways to do this.

In [ ]:
analysis_model = model

Replace `model` by `dropout_model` in case you want to analyze the latter.

### Required imports

In [ ]:
import itertools
import pandas as pd
import seaborn as sns

### Confusion matrix

The model produces some classification errors, it would be interesting to see the type of errors.  Computing a confustion matrix is useful for that prupose.

In [ ]:
y_predict_classes = analysis_model.predict_classes(x_test)

In [ ]:
y_test_classes = np.argmax(y_test, axis=1)

In [ ]:
cm = confusion_matrix(y_test_classes, y_predict_classes)
cm

From the confusion matrix, it is clear that, e.g., 2 and 7 get confused, as well as 4 and 9.  Given the data, that should come as no surprise.

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          cmap=plt.cm.Blues):

    log1p_cm = np.log1p(cm)
    if normalize:
        cm = cm.astype(np.float)/cm.sum(axis=1)[:, np.newaxis]
    
    figure, axes = plt.subplots(figsize=(6, 6))
    axes.imshow(log1p_cm, interpolation='nearest', cmap=cmap)
    axes.set_xticks(classes)
    axes.set_yticks(classes)

    fmt = '{0:.4f}' if normalize else '{0:d}'
    thresh = 0.5*cm.max()
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        axes.text(j, i, fmt.format(cm[i, j]),
                  horizontalalignment="center",
                  color="white" if cm[i, j] > thresh else "black",
                  fontsize=8)

    figure.tight_layout()
    axes.set_ylabel('True label')
    axes.set_xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(cm, range(10), normalize=True)

### Sensitivity to initial conditions

How sensitive is the training process to the initial values of the model?  Let's train and evaluate the model several times, and observe the outcomes.

In [ ]:
np.random.seed(1234)
tf.set_random_seed(459845)
names = ('train', 'val', 'test')
inputs = (x_train, x_val, x_test)
outputs = (y_train, y_val, y_test)
losses = dict()
accuracies = dict()
for name in names:
    losses[name] = []
    accuracies[name] = []
for i in range(10):
    print(f'training model {i+ 1:d}')
    mod = Sequential()
    mod.add(Dense(512, activation='relu', input_shape=(784,)))
    mod.add(Dropout(0.2))
    mod.add(Dense(512, activation='relu'))
    mod.add(Dropout(0.2))
    mod.add(Dense(10, activation='softmax'))
    mod.compile(loss='categorical_crossentropy', optimizer=SGD(),
                metrics=['accuracy'])
    _ = mod.fit(x_train, y_train, batch_size=128, epochs=100,
                verbose=0, validation_data=(x_val, y_val))
    for name, input, output in zip(names, inputs, outputs):
        loss, accuracy = mod.evaluate(input, output)
        losses[name].append(loss)
        accuracies[name].append(accuracy)
    mod.save(f'mnist_mpl_{i + 1:02d}.h5')

Losses and accuracies are quite reproducible across runs with different initializations of the model.

In [ ]:
sns.stripplot(data=pd.DataFrame(losses), jitter=0.1);

In [ ]:
sns.stripplot(data=pd.DataFrame(accuracies), jitter=0.1);